In [1]:
import pandas as pd

In [13]:
# Get the FRED mortgage data to be merged with the Zillow data
fred_csv_path = '../Data/FRED/clean_mortgage_data_iso_date.csv'
fred_df = pd.read_csv(fred_csv_path)

In [3]:
# Set the path for the U.S. Bureau of Labor Statistics (BLS) by city data to be merged with the Zillow data
bls_csv_path = '../Data/BLS/clean/'

In [6]:
# Set up all Zillow files and resulting column names
base_read_path = '../Data/Zillow/new_metro/clean/'
zillow_file_name_to_col_name_map: dict[str, str] = {
    'city_mean_sale_price_iso.csv': 'MeanSalePrice',
    'city_sales_count_iso.csv': 'SalesCount',
    'city_total_transaction_value_iso.csv': 'TotalTransactionValue',
    'city_home_value_idx_iso.csv': 'HomeValueIndex',
    'city_home_value_idx_sm_sa_iso.csv': 'HomeValueIndexSmSa',
    
    'city_market_temp_index_iso.csv': 'MarketTempIdx',
    'city_mean_days_to_close_iso.csv': 'DaysToClose',
    'city_mean_doz_pending_iso.csv': 'DaysOnZillow',
    'city_mean_listings_price_cut_amt_iso.csv': 'ListingPriceCutAmt',
    'city_mean_listings_price_cut_pct_iso.csv': 'ListingPriceCutPct',
    'city_mean_sale_to_list_iso.csv': 'SaleToList',
    'city_new_con_mean_sale_price_iso.csv': 'NewConstructMeanSalePrice',
    'city_new_con_sales_count_iso.csv': 'NewConstructSalesCount',
    'city_pct_listings_price_cut_iso.csv': 'PctListingsPriceCut',
    'city_pct_sold_above_list_iso.csv': 'PctSoldAboveList',
    'city_pct_sold_below_list_iso.csv': 'PctSoldBelowList',
}

In [14]:
# Read all Zillow files into dataframes and store in a dictionary
df_dict: dict[str, pd.DataFrame] = {}
for file_name, col_name in zillow_file_name_to_col_name_map.items():
    df = pd.read_csv(base_read_path + file_name)
    df_dict[col_name] = df

In [15]:
# NOTE: Buffalo NY doen't have data in all files so usineg CHARLOTTE instead.
# Limit our scope of cities to Denver, Buffalo, Cincinnati, Columbus, Indianapolis, and Providence
selected_city_names = ['DENVER, CO', 'CHARLOTTE, NC', 'CINCINNATI, OH', 'ATLANTA, GA', 'INDIANAPOLIS, IN', 'PROVIDENCE, RI', 'TAMPA, FL']
city_df_dict: dict[str, list[pd.DataFrame]] = {}
for city_name in selected_city_names:
    city_df_dict[city_name] = []
    for col_name, df in df_dict.items():
        # print(city_name, col_name)
        temp_df = df.loc[df['RegionName'] == city_name]
        temp_df = temp_df.drop(columns=['RegionName'])
        temp_df = temp_df.transpose()
        temp_df.reset_index(inplace=True)
        temp_df.columns = ['DATE', col_name]
        city_df_dict[city_name].append(temp_df)

In [16]:
# Now merge all of the Zillow dataframes for each city into one dataframe with appropriate columns
city_merged_df_dict: dict[str, pd.DataFrame] = {}
for city_name in selected_city_names:
    merged_df = city_df_dict[city_name][0]
    for i in range(1, len(city_df_dict[city_name])):
        merged_df = pd.merge(merged_df, city_df_dict[city_name][i], on=['DATE'], how='left')
    
    # Add the FRED mortgage data
    merged_df = pd.merge(merged_df, fred_df, on=['DATE'], how='left')
    
    # Add the U.S. Bureau of Labor Statistics (BLS) by city data
    short_city_name = city_name.split(',')[0].upper()
    bls_df = pd.read_csv(bls_csv_path + short_city_name + '_emp.csv')
    bls_df = bls_df.drop(columns=['CITY_NAME'])
    merged_df = pd.merge(merged_df, bls_df, on=['DATE'], how='left')
    
    # Add the CityName
    merged_df['CityName'] = [city_name] * len(merged_df)
    
    # Save merged dataframe to dictionary
    city_merged_df_dict[city_name] = merged_df    

In [17]:
# Sanity check for Denver
df = city_merged_df_dict['DENVER, CO']
df


,DATE,MeanSalePrice,SalesCount,TotalTransactionValue,HomeValueIndex,HomeValueIndexSmSa,MarketTempIdx,DaysToClose,DaysOnZillow,ListingPriceCutAmt,...,NewConstructSalesCount,PctListingsPriceCut,PctSoldAboveList,PctSoldBelowList,MORTGAGE15US,MORTGAGE30US,RATE_DIFFERENCE,NUM_JOBS,GROWTH_PCT,CityName
0,2008-04-30,245069.0,4305.0,9.275848e+08,239786.727396,243635.279120,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.465,5.918,0.452,1259.0,1.0,"DENVER, CO"
1,2008-05-31,242919.0,4779.0,1.043794e+09,240755.620219,242719.130794,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.600,6.036,0.436,1256.8,-1.0,"DENVER, CO"
2,2008-06-30,245529.0,4789.0,1.136320e+09,241296.223700,241952.165191,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.910,6.320,0.410,1258.2,-0.6,"DENVER, CO"
3,2008-07-31,247607.0,4922.0,1.195890e+09,240843.494803,240801.464621,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.972,6.426,0.454,1256.1,-1.0,"DENVER, CO"
4,2008-08-31,246384.0,4333.0,1.154412e+09,239424.931087,239672.270501,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.025,6.478,0.452,1256.2,-0.2,"DENVER, CO"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,2024-02-29,635082.0,2767.0,1.507124e+09,572034.304896,588450.438544,74.0,28.0,44.0,16524.040586,...,368.0,0.196883,0.257586,0.497816,6.102,6.776,0.674,NaN,NaN,"DENVER, CO"
191,2024-03-31,654724.0,3327.0,1.776480e+09,580597.558905,588917.680727,72.0,28.0,36.0,17538.582027,...,439.0,0.222357,0.315551,0.441306,6.175,6.820,0.645,1627.7,0.5,"DENVER, CO"
192,2024-04-30,682101.0,3660.0,2.225136e+09,588155.609179,590109.794947,68.0,28.0,28.0,18508.070754,...,391.0,0.247789,0.365956,0.386085,6.263,6.992,0.730,1621.2,-1.1,"DENVER, CO"
193,2024-05-31,695652.0,4050.0,2.560065e+09,591315.674562,590809.175996,62.0,28.0,24.0,18943.332922,...,374.0,0.285580,0.387427,0.371205,6.346,7.060,0.714,1623.4,-1.0,"DENVER, CO"


In [18]:
# Save merged dataframes to csv
for city_name, df in city_merged_df_dict.items():
    short_city_name = city_name.split(',')[0].upper()
    df.to_csv(f'../Data/Clean/combined/{short_city_name.capitalize()}_combined.csv', index=False)
